In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

cobranza_path = 'BBVACF - Informe Cobranzas 10-10-2024.xlsx'
df_cobranza = pd.read_excel(cobranza_path)
df_cobranza.shape

(18830, 32)

In [2]:
df_test = df_cobranza.copy()

def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('  ', ' ').replace('  ', ' ').replace('.', '_').replace(' ', '_').upper() for column in columns_list]

df_test.columns = clean_columns(df_test.columns)

cols_req = ['DOI', 'NOMBRE_CLIENTE', 'NUMERO_DE_CONTRATO', 'NUMERO_CUOTA', 'PLAZO', 
            'MONEDA', 'INTERES', 'INT_COMP_MORA', 'INT_MORATORIO', 'AMORTIZACION', 
            'SEG_VEHICULAR', 'SEG_DESGRAVAMEN', 'COMISION', 'COMISION_COBRANZA', 'COMISION_ENVIO_DOC', 
            'PENALIDAD_POR_MORA', 'MONTO_TOTAL_CUOTA', 'NUMERO_DE_TELEFONO_1', 'NUMERO_DE_TELEFONO_2', 'CORREO']
df_test = df_test[cols_req] 

df_test.rename(columns={
    'NUMERO_DE_CONTRATO': 'CONTRATO',
    'PENALIDAD_POR_MORA': 'PENALIDAD',
    'MONTO_TOTAL_CUOTA': 'DEUDA_VENCIDA_TOTAL',
    'NUMERO_DE_TELEFONO_1': 'TELEFONO_1', 
    'NUMERO_DE_TELEFONO_2': 'TELEFONO_2',
    }, inplace=True)

df_test['CAPITAL_VENCIDO'] = df_test['DEUDA_VENCIDA_TOTAL'] - (df_test['INTERES'] + df_test['INT_COMP_MORA'] 
                                                                + df_test['INT_MORATORIO']  + df_test['SEG_DESGRAVAMEN'] 
                                                                + df_test['SEG_VEHICULAR'] + df_test['COMISION'] 
                                                                + df_test['COMISION_COBRANZA'] + df_test['COMISION_ENVIO_DOC'] 
                                                                + df_test['PENALIDAD'])

df_test['CAPITAL_VENCIDO'] = df_test['CAPITAL_VENCIDO'].apply(lambda x: round(x, 2))
df_test['DOI'] = df_test['DOI'].astype(str).str.strip().str.replace(' ', '')
df_test['DOI'] = df_test['DOI'].astype('Int64').astype(str).apply(lambda x: x if len(x) == 11 else str(x).zfill(8))
df_test['CONTRATO'] = df_test['CONTRATO'].astype('Int64').astype(str).str.zfill(8)
df_test['CORREO'] = df_test['CORREO'].astype(str).str.strip().replace(' ', '').str.upper()
df_test['MONEDA'] = df_test['MONEDA'].astype(str).str.strip().replace(' ', '').str.upper()
df_test['COMISIONES'] = df_test['COMISION'] + df_test['COMISION_COBRANZA'] + df_test['COMISION_ENVIO_DOC']
df_test['SEGUROS'] = df_test['SEG_DESGRAVAMEN'] + df_test['SEG_VEHICULAR']
df_test['DEUDA_VENCIDA_DESCUENTO'] = df_test['DEUDA_VENCIDA_TOTAL'] - (df_test['INT_COMP_MORA'] + df_test['PENALIDAD'])
df_test['TELEFONO_1'] = df_test['TELEFONO_1'].astype(str).str.strip().replace(' ', '')
df_test['TELEFONO_2'] = df_test['TELEFONO_2'].astype(str).str.strip().replace(' ', '')
df_test['TELEFONO'] = df_test.apply(lambda x: x['TELEFONO_2'] if len(x['TELEFONO_2']) == 9 else x['TELEFONO_1'], axis=1)
df_test['NUMERO_CUOTA_MIN'] = df_test.groupby('CONTRATO')['NUMERO_CUOTA'].transform('min')
df_test['NUMERO_CUOTA_MAX'] = df_test.groupby('CONTRATO')['NUMERO_CUOTA'].transform('max')
df_test['CUOTAS_PAGADAS'] = df_test['NUMERO_CUOTA_MIN'] - 1
df_test['CUOTAS_VENCIDAS'] = df_test['NUMERO_CUOTA_MAX'] - df_test['NUMERO_CUOTA_MIN'] + 1
df_test['CUOTAS_POR_VENCER'] = df_test['PLAZO'] - df_test['NUMERO_CUOTA_MAX']

cols_final = ['DOI', 'CONTRATO', 'NOMBRE_CLIENTE', 'TELEFONO', 'CORREO', 'MONEDA', 'CUOTAS_PAGADAS', 
                'CUOTAS_VENCIDAS', 'CUOTAS_POR_VENCER', 'PLAZO', 'INTERES', 'INT_COMP_MORA', 'INT_MORATORIO', 
                'SEGUROS', 'COMISIONES', 'PENALIDAD', 'CAPITAL_VENCIDO', 'DEUDA_VENCIDA_TOTAL', 'DEUDA_VENCIDA_DESCUENTO', ]
df_test = df_test[cols_final]

df_test_grouped = df_test.groupby(['DOI', 'CONTRATO']).agg({
    'NOMBRE_CLIENTE': 'first',
    'TELEFONO': 'first',
    'CORREO': 'first',
    'MONEDA': 'first',
    'CUOTAS_PAGADAS': 'first',
    'CUOTAS_VENCIDAS': 'first',
    'CUOTAS_POR_VENCER': 'first',
    'PLAZO': 'max',
    'INTERES': 'sum',
    'INT_COMP_MORA': 'sum',
    'INT_MORATORIO': 'sum',
    'SEGUROS': 'sum',
    'COMISIONES': 'sum',
    'PENALIDAD': 'sum',
    'CAPITAL_VENCIDO': 'sum',
    'DEUDA_VENCIDA_TOTAL': 'sum',
    'DEUDA_VENCIDA_DESCUENTO': 'sum',
}).reset_index()

df_test_grouped.sort_values(by=['CUOTAS_POR_VENCER', 'DOI', 'CONTRATO'], inplace=True)
df_test_grouped.reset_index(drop=True, inplace=True)
df_test_grouped.to_excel('CONSOLIDADO_INFORME_COBRANZA.xlsx', index=False)
df_test_grouped.head(5)

,DOI,CONTRATO,NOMBRE_CLIENTE,TELEFONO,CORREO,MONEDA,CUOTAS_PAGADAS,CUOTAS_VENCIDAS,CUOTAS_POR_VENCER,PLAZO,INTERES,INT_COMP_MORA,INT_MORATORIO,SEGUROS,COMISIONES,PENALIDAD,CAPITAL_VENCIDO,DEUDA_VENCIDA_TOTAL,DEUDA_VENCIDA_DESCUENTO
0,00367706,00498851,ARMENGOL OYOLA FLORES,997851328,GOLSPORT_27@HOTMAIL.COM,SOLES,0,1,0,1,19970.59,28690.81,0,601.17,320,80,13698.66,63361.23,34590.42
1,00452751,00329459,DENISTON CLEMENTE VIZCARRA ESPINOZA,992149345,VIZMARPRODUCE@GMAIL.COM,SOLES,39,21,0,60,2100.70,4344.19,0,3483.90,0,0,17774.10,27702.89,23358.70
2,00514658,00555918,RENZA LOURDES GAMBETTA QUELOPANA,953687328,PORVERIFICAR@HOTMAIL.COM,SOLES,11,1,0,12,0.00,0.00,0,57.62,0,0,63670.70,63728.32,63728.32
3,00536850,00463352,RAUL ADRIAN TEJERA SILVA,982198667,RAULTEJERA@GMAIL.COM,SOLES,13,35,0,48,5481.18,9078.38,0,5428.15,0,0,21209.98,41197.69,32119.31
4,00824761,00350089,THALIA ABAD ALVARADO,922301284,THALIAABAD_2014@HOTMAIL.COM,SOLES,29,31,0,60,4497.52,7636.94,0,4329.46,0,0,26782.71,43246.63,35609.69


In [3]:
df_base = pd.read_excel('BASES/CUADRO TOTAL CONSOLIDADO.xlsx')
df_base['DOI'] = df_base['DOI'].astype(str).str.strip().str.replace(' ', '')
df_base['DOI'] = df_base['DOI'].astype('Int64').astype(str).apply(lambda x: x if len(x) == 11 else str(x).zfill(8))
df_base['CONTRATO'] = df_base['CONTRATO'].astype('Int64').astype(str).str.zfill(8)
df_base.columns = clean_columns(df_base.columns)
df_base.drop(columns=['NOMBRE'], inplace=True)
df_base.shape

(192, 2)

In [4]:
df_cruce = pd.merge(df_base, df_test_grouped, on=['DOI', 'CONTRATO'], how='left')
df_cruce['CUOTAS_PAGADAS'] = df_cruce['CUOTAS_PAGADAS'].astype('Int64')
df_cruce['CUOTAS_VENCIDAS'] = df_cruce['CUOTAS_VENCIDAS'].astype('Int64')
df_cruce['CUOTAS_POR_VENCER'] = df_cruce['CUOTAS_POR_VENCER'].astype('Int64')
df_cruce['PLAZO'] = df_cruce['PLAZO'].astype('Int64')

df_cuotas_vencidas = df_cruce[df_cruce['CUOTAS_POR_VENCER'] == 0]
df_cuotas_por_vencer = df_cruce[df_cruce['CUOTAS_POR_VENCER'] > 0]
df_cuotas_vencidas.sort_values(by=['DOI', 'CONTRATO'], inplace=True)
df_cuotas_por_vencer.sort_values(by=['DOI', 'CONTRATO'], inplace=True)

with pd.ExcelWriter('CUADRO_TOTAL_CONSOLIDADO.xlsx') as writer:
    df_cuotas_vencidas.to_excel(writer, sheet_name='BASE VENCIDAS', index=False)
    df_cuotas_por_vencer.to_excel(writer, sheet_name='BASE POR VENCER', index=False)